In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [5]:
def f(series):
    return series.shift(1).cumsum().fillna(0) / np.arange(len(series))


In [6]:
df["target_enc_content_id"] = df.groupby("content_id")["answered_correctly"].transform(f)

In [7]:
df["target_enc_user_id"] = df.groupby("user_id")["answered_correctly"].transform(f)

In [8]:
df["user_count"] = df.groupby("user_id").cumcount()

In [9]:
df_question = pd.read_csv("../input/riiid-test-answer-prediction/questions.csv")

In [10]:
df_question

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125


In [14]:
def tags(df):
    tag = df["tags"].str.split(" ", n=10, expand=True)
    tag.columns = [f"tags{i}" for i in range(1, len(tag.columns) + 1)]

    for col in ["tags1", "tags2", "tags3", "tags4", "tags5", "tags6"]:
        if col in tag.columns:
            df[col] = pd.to_numeric(tag[col], errors='coerce').fillna(-1).astype("int16")
        else:
            df[col] = -1
            df[col].astype("int16")

In [15]:
tags(df_question)

In [21]:
w_df_q = df_question[df_question["part"] == 5]
w_df_q["tags"].value_counts()

8      697
73     512
53     417
1      348
96     330
55     287
79     272
14     211
173    196
89     191
109    177
91     113
60     103
116     78
72      68
134     61
7       61
133     58
125     56
166     55
48      51
152     50
52      50
147     49
181     45
4       43
45      42
66      41
64      40
156     40
174     39
159     37
168     36
80      32
177     32
25      27
123     26
47      25
28      24
85      23
170     23
54      23
180     22
128     22
75      21
108     20
26      20
175     19
65      18
44      18
33      18
115     17
127     17
43      16
24      16
95      15
151     13
15      12
182     12
49      12
124     12
167     11
179     10
31      10
57      10
132     10
112     10
58      10
23      10
172      9
78       6
34       6
Name: tags, dtype: int64

In [23]:
w_df_q = df_question[df_question["part"] == 2]
w_df_q["tags1"].value_counts()

143    664
90     178
155    148
62     140
2      137
137     96
148     85
138     79
129     50
69      31
17      30
32       4
100      3
41       1
38       1
Name: tags1, dtype: int64

In [24]:
w_df_q = df_question[df_question["part"] == 3]
w_df_q["tags1"].value_counts()

136    533
74     283
82     276
106    195
157    123
113     86
161     66
Name: tags1, dtype: int64